In [1]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

# Code to clean the data

In [2]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_70690/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [3]:
# data.iloc[0]

Removing useless fields

In [4]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [5]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")

In [6]:
#TODO: "Rome" non viene preso, quindi risolvi questa cosa
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows()]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



  0%|          | 0/224763 [00:00<?, ?it/s]

In [7]:
final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

In [8]:
df = final_df.reset_index()

### Save the dataframe as a zipped file

In [9]:
# from zipfile import ZipFile
# import os

# DF_PATH = "./italian_restaurant_clean.csv"
# ZIP_PATH = "./italian_restaurant_clean.zip"
# final_df.to_csv(DF_PATH)
# with ZipFile(ZIP_PATH, "w") as myzip:
#     myzip.write(DF_PATH)

# os.remove(DF_PATH)

# Code to insert the restaurants in the database 

In [10]:
# df = pd.read_csv("./italian_restaurant_clean.zip")

In [23]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the Universal Sentence Encoder
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/")
def embed_name(name):
    embeddings = model(['my text', 'batch'])
    return embeddings[0]

FileNotFoundError: Op type not registered 'SentencepieceOp' in binary running on MacBook-Air-di-Domiziano.local. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [ ]:
restaurant1 = "Mariuccio"
restaurant2 = "L'Ortal"

# Embed the restaurant names
embedding1 = embed_name(restaurant1)
embedding2 = embed_name(restaurant2)

# Compute similarity between the embeddings
distance = 1 - np.inner(embedding1, embedding2)

print(distance)

TypeError: pruned(dense_shape, indices, values) takes 0 positional arguments, got 1.

In [24]:
from minio import Minio

URL = 'http://localhost:3000/restaurants/create-restaurant'

client = Minio(
    "localhost:9002",
    access_key="rxSdZSDLIM5nJcr1",
    secret_key="ATjj0yffTeQqIAgVC7GcxONoNGT0l7hS",
    secure=False  # Otherwise bug
)

In [25]:
def get_num_images():
    objects = client.list_objects(bucket_name="images")
    num_images = sum(1 for _ in objects)
    return num_images

num_images = get_num_images()

def assign_image(index):
    return index % num_images

def create_restaurant(row):
    body = {
            "name": row["restaurant_name"],
            "embeddingName": f"embedding_{(row['index'])}",
            "imageName": f"restaurant_image_{assign_image(row['index'])}",
            "address": row["address"],
            "latitude": float(row["latitude"]),
            "longitude": float(row["longitude"]),
            "country": row["country"],
            "region": row["region"],
            "province": row["province"],
            "city": row["city"],
            "tags": row["top_tags"],
            "cuisines": row["cuisines"],
            "specialDiets": row["special_diets"],
            "priceLevel": row["price_level"],
            "meals": row["meals"],
            "avgRating": float(row["avg_rating"]),
            "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
            "veganFriendly": row["vegan_options"] == "Y",
            "glutenFree": row["gluten_free"] == "Y",
            "reviewsNumber": int(row["total_reviews_count"])
        }

    
    result = requests.post(URL, json=body)

import json
import io
def insert_embeddings(row):
    BUCKET_NAME = "embeddings"
    embedding =  embed_name(row["restaurant_name"]).numpy().tolist()
    embedding_str = json.dumps(embedding)
    embedding_bytes = embedding_str.encode("utf-8")
    client.put_object(
            BUCKET_NAME, f"embedding_{row['index']}", io.BytesIO(embedding_bytes), length=len(embedding_bytes))

tqdm.pandas()
df.progress_apply(create_restaurant, axis=1)
# df.progress_apply(insert_embeddings, axis=1)

  0%|          | 0/222603 [00:00<?, ?it/s]

KeyboardInterrupt: 